In [60]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools import tool
from langchain.schema.runnable import RunnablePassthrough
from pydantic import BaseModel, Field
from typing import List, Optional
import requests
from langchain.schema.agent import AgentFinish

In [33]:
class productDetails(BaseModel):
    product_category: str = Field(description="The name of the product required")
    product_id : int = Field(description="The id of the desired product to look for ")
class DiscountDetails(BaseModel):
    min_discount: float = Field(description="The minmum discount price of the product to look for")
    max_discount : float = Field(description="The maxmum discount price of the product to look for ")

In [35]:
@tool(args_schema=productDetails)
def get_product_information(product_category:str, product_id:int) ->dict:
     """get details of a given product name of the given url"""
     url = "http://localhost/ecommerce_data/data.json"
     response = requests.get(url)
     if response.status_code ==200:
          data= response.json()
          for product in data[product_category]:
               if product['id'] == product_id:
                    return product
               return {"error": "Product not found"}
     else:
          raise Exception(f"API Request failed with status code: {response.status_code}")

In [71]:
@tool(args_schema=DiscountDetails)
def get_discounted_products(min_discount:float, max_discount:float) -> dict:
    """Search for all products that have discounts available"""
    url = "http://localhost/ecommerce_data/data.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        discounted_products = []
        for product in data['discountItems']:
            if min_discount <= float(product['price']) <= max_discount:
                discounted_products.append({
                    
                    'name': product['name'],
                    'price': product['price'],
                    'cover': product['cover']
                })
        return discounted_products
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")


In [45]:
from langchain.tools.render import format_tool_to_openai_function

functions = [
   format_tool_to_openai_function(get_product_information), 
   format_tool_to_openai_function(get_discounted_products)
]

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_12708\67842453.py:4: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_product_information),


In [49]:
model.invoke("how many products have discount between 100 to 200?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"min_discount":100,"max_discount":200}', 'name': 'get_discounted_products'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2a1e6267-745c-45ec-8075-3f925af9b2f6-0')

In [ ]:
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt =ChatPromptTemplate.from_messages([
    ("system", "Your a helpful intelligent and sassy business assistant for Rhodrick that sells differents products"),
    ("user", "{input}"),
])

In [61]:

def route_to(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "get_discounted_products": get_discounted_products, 
            "get_product_information": get_product_information,
        }
        return tools[result.tool].run(result.tool_input)

In [62]:
chain = prompt |model|OpenAIFunctionsAgentOutputParser()|route_to


In [74]:
chain.invoke({"input":"show me products with discount  100"})

[{'name': 'Setgearr 2022',
  'price': '100',
  'cover': 'http://169.254.228.82/ecommerce_data/images/discount/discount-4.png'}]